In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import time
import sqlalchemy
from sqlalchemy import create_engine, Column, Integer, String, PrimaryKeyConstraint, func,inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session
import sqlite3
from sqlalchemy.ext.automap import automap_base


browser = Browser('chrome')
url = 'https://www.pickleheads.com/courts/us/florida'
browser.visit(url)
buttons = browser.find_by_css(".css-2ttlle")
button_index_to_click = 1
if button_index_to_click < len(buttons):
    button_to_click = buttons[button_index_to_click]
    button_to_click.click()
    time.sleep(2)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [2]:
whole_list=soup.find('div',class_='css-12eoos1')

In [3]:
location = []
city = []
courts=[]

for entry in whole_list:
    location_data = entry.find('span', class_='css-uyrttv')
    if location_data:
        location.append(location_data.text)
    else:
        continue

    city_data = entry.find(class_='css-1vl3ez2')
    
    if city_data:
        city.append(city_data.text)

    court_data=entry.select('.css-1vl3ez2')
    if court_data: 
        last_span=court_data[-1] 
        courts.append(last_span.text)

In [13]:
#Create a dataframe with the locations and cities.
pickleball_dict={"Location Name":location,
                 "City":city,
                 "Number of Courts":courts}
fl_pickleball_df=pd.DataFrame(pickleball_dict)
fl_pickleball_df_sorted=fl_pickleball_df.sort_values(by='City',ascending=True)
city_county_df=pd.read_csv('Florida City County and Coordinates.csv')
city_county_df=city_county_df.rename(columns={"CITY":"City","COUNTY":"County","LATITUDE":"Latitude","LONGITUDE":"Longitude"})

In [14]:
fl_full_info=pd.merge(fl_pickleball_df_sorted,city_county_df,on='City',how='left')
fl_full_info.to_csv('pickleball_courts_with_counties_updated.csv',index=False)
fl_full_info['Number of Courts'] = fl_full_info['Number of Courts'].str.replace(' courts', '')
fl_full_info['Number of Courts'] = fl_full_info['Number of Courts'].str.replace(' court', '')
fl_full_info['ID'] = range(1, len(fl_full_info) + 1)
fl_full_info.head()


,Location Name,City,Number of Courts,County,Latitude,Longitude,ID
0,Hal Brady Recreation Complex,Alachua,6,Alachua,29.796996,-82.480531,1
1,Eastmonte Park,Altamonte Springs,4,Seminole,28.662728,-81.371908,2
2,Sanlando Park,Altamonte Springs,10,Seminole,28.662728,-81.371908,3
3,Westmonte Park,Altamonte Springs,3,Seminole,28.662728,-81.371908,4
4,Lake Brantley South Gym,Altamonte Springs,2,Seminole,28.662728,-81.371908,5


In [19]:
conn = sqlite3.connect('florida_info_db.sqlite')
fl_full_info.to_sql('court_location', conn, index=False, if_exists='replace', dtype={'ID': 'INTEGER PRIMARY KEY'})

1033

In [9]:
browser.quit()

In [ ]:
# Import the dependencies.
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func,inspect


#################################################
# Database Setup
#################################################

engine = create_engine("sqlite:///fl_full_db.sqlite")

# reflect an existing database into a new model
Base=automap_base()
# reflect the tables
Base.prepare(autoload_with=engine)

# Save references to each table
court_locations=Base.classes.keys()
print(court_locations)

['court_locations']
